# Debug Runt Transform 'direod'

In [1]:
import sys
import os
from os import sep
from os.path import dirname, realpath
from pathlib import Path
import logging
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which is why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname
    
def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fix_path(get_cwd('test.ipynb', 'recon' +sep))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numba import jit, vectorize
from dask import delayed, dataframe as dd

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 100)
pd.set_option('display.max_columns', 50)

from common_util import MUTATE_DIR, DT_HOURLY_FREQ, DT_CAL_DAILY_FREQ
from common_util import load_json, dump_json, remove_dups_list, list_get_dict, is_empty_df, search_df, benchmark
from data.data_api import DataAPI
from data.access_util import df_getters as dg, col_subsetters2 as cs2
from mutate.common import default_runt_dir_name, default_trfs_dir_name
from mutate.runt_util import *
from mutate.runt import fill_defaults, get_variants, get_row_mask_keychain, get_desc_pfx, make_runt_entry

DEBUG:matplotlib:$HOME=/home/kevin
DEBUG:matplotlib:matplotlib data path /home/kevin/miniconda3/lib/python3.6/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:loaded rc file /home/kevin/miniconda3/lib/python3.6/site-packages/matplotlib/mpl-data/matplotlibrc
DEBUG:matplotlib:matplotlib version 2.2.3
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_weakrefset', '_bootlocale', '_locale', 'site', 'os', 'errno', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'sysconfig', '_sysconfigdata_m_linux_x86_64-linux-gnu', 'types', 'functools', '_functools', 'collections', 'operator', '_operator', 'keyword', 'heapq

DEBUG:matplotlib:CACHEDIR=/home/kevin/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/kevin/.cache/matplotlib/fontList.json
DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown
DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown


## General

### Set Logging Level and Date Range

In [2]:
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
date_range = {
    'id': ('lt', 2018)
}

### Set Runt and Transform Directories

In [3]:
runt_dir_name = default_runt_dir_name
trfs_dir_name = default_trfs_dir_name

runt_dir = MUTATE_DIR +runt_dir_name
trfs_dir = runt_dir +trfs_dir_name

### Load all transforms

In [4]:
trf_defaults = load_json('trf_defaults.json', dir_path=runt_dir)
graph = load_json('graph.json', dir_path=runt_dir)
trfs = {}

logging.info('loading step settings...')
for fname in os.listdir(trfs_dir):
    trf = load_json(fname, dir_path=trfs_dir)
    trfs[trf['meta']['name']] = trf

INFO:root:loading step settings...


### Define Test Step Function

In [5]:
def test_step(step_info, date_range, overwrites = {}):
    meta, fn, var, rm, src, dst = step_info['meta'], step_info['fn'], step_info['var'], step_info['rm'], step_info['src'], step_info['dst']

    # Loading transform, apply, and frequency settings
    ser_fn = overwrites['ser_fn'] if ('ser_fn' in overwrites) else RUNT_FN_TRANSLATOR[fn['ser_fn']]
    rtype_fn = overwrites['df_fn'] if ('df_fn' in overwrites) else RUNT_TYPE_TRANSLATOR[fn['df_fn']]
    col_fn = RUNT_NMAP_TRANSLATOR[fn['col_fn']]
    freq = overwrites['freq'] if ('freq' in overwrites) else RUNT_FREQ_TRANSLATOR[fn['freq']]
    res_freq = RUNT_FREQ_TRANSLATOR[meta['res_freq']]

    # Making all possible parameter combinations
    variants = get_variants(var, meta['var_fmt'])

    # Loading row mask, if any
    if (rm is not None):
        rm_dg, rm_cs = list_get_dict(dg, rm), list_get_dict(cs2, rm)
        rm_paths, rm_recs, rm_dfs = DataAPI.load_from_dg(rm_dg, rm_cs)
        rm_keys = [remove_dups_list([key_chain[i] for key_chain in rm_paths]) for i in range(len(rm_paths[0]))]

    # Loading input data
    src_dg, src_cs = list_get_dict(dg, src), list_get_dict(cs2, src)
    src_paths, src_recs, src_dfs = DataAPI.load_from_dg(src_dg, src_cs)
    logging.debug('src_paths[0] ' +str(src_paths[0]))
    logging.debug('src_paths[-1] ' +str(src_paths[-1]))

    # Run transforms on inputs
    for key_chain in src_paths:
        logging.info('data: ' +str('_'.join(key_chain)))
        src_rec, src_df = list_get_dict(src_recs, key_chain), list_get_dict(src_dfs, key_chain)
        src_df = src_df.loc[search_df(src_df, date_range), :].dropna(axis=0, how='all')

        # Masking rows in src from row mask
        if (rm is not None):
            rm_key_chain = get_row_mask_keychain(key_chain, rm_keys)
            rm_df = list_get_dict(rm_dfs, rm_key_chain).dropna()
            not_in_src = rm_df.index.difference(src_df.index)
            logging.debug('row mask: ' +str('_'.join(rm_key_chain)))
            if (len(not_in_src)>0):
                logging.debug('rm_idx - src_idx: ' +str(not_in_src))
                src_df = src_df.loc[src_df.index & rm_df.index, :].dropna(axis=0, how='all')
            else:
                src_df = src_df.loc[rm_df.index, :].dropna(axis=0, how='all')

        logging.debug('pre_transform: ' +str(src_df))

        # Running variants of the transform
        for variant in variants:
            fn = ser_fn(**variant)
            runted_df = rtype_fn(src_df, ser_fn(**variant), freq, col_fn(**variant))
            desc_sfx = meta['rec_fmt'].format(**variant)
            desc_pfx = get_desc_pfx(key_chain, src_rec)
            desc = '_'.join([desc_pfx, desc_sfx])

            if (meta['mtype_from']=='name'):       mutate_type = meta['name']
            elif (meta['mtype_from']=='rec_fmt'):  mutate_type = desc_sfx

            assert(not is_empty_df(runted_df))
            entry = make_runt_entry(desc, res_freq, mutate_type, src_rec)
            logging.info('dumping ' +desc +'...')
            logging.debug('post_transform: ' +str(runted_df))
            yield entry, runted_df

## Transform Debug

### Set Transform
Fill it in (default is day moving average)

In [6]:
trf_name = 'direod'

### Imports

### Test Section

In [7]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [8]:
trf = trfs[trf_name]
step = fill_defaults(trf, trf_defaults)

In [11]:
for runt_rec, runt_df in test_step(step, date_range):
    None

DEBUG:root:src_paths[0] ['dow_jones', 'reteod', 'raw_pba_oc_retxeod_reteod']
DEBUG:root:src_paths[-1] ['russell_2000', 'reteod', 'raw_vol_lh_retxeod_reteod']
INFO:root:data: dow_jones_reteod_raw_pba_oc_retxeod_reteod
DEBUG:root:pre_transform:                              pba_oc
id                                 
1998-01-02 00:00:00+00:00  0.006928
1998-01-05 00:00:00+00:00  0.001500
1998-01-06 00:00:00+00:00 -0.009025
1998-01-07 00:00:00+00:00 -0.012858
1998-01-08 00:00:00+00:00 -0.012579
1998-01-09 00:00:00+00:00 -0.028136
1998-01-12 00:00:00+00:00  0.009969
1998-01-13 00:00:00+00:00  0.011142
1998-01-14 00:00:00+00:00  0.006798
1998-01-15 00:00:00+00:00 -0.011588
1998-01-16 00:00:00+00:00  0.006956
1998-01-20 00:00:00+00:00  0.015454
1998-01-21 00:00:00+00:00 -0.009529
1998-01-22 00:00:00+00:00 -0.008023
1998-01-23 00:00:00+00:00 -0.003802
1998-01-26 00:00:00+00:00  0.001456
1998-01-27 00:00:00+00:00  0.012588
1998-01-28 00:00:00+00:00  0.012846
1998-01-29 00:00:00+00:00  0.007112
1

INFO:root:dumping raw_pba_oc_retxeod_reteod_direod(2%)...
DEBUG:root:post_transform:                            pba_oc
id                               
1998-01-02 00:00:00+00:00     0.0
1998-01-05 00:00:00+00:00     0.0
1998-01-06 00:00:00+00:00     0.0
1998-01-07 00:00:00+00:00    -1.0
1998-01-08 00:00:00+00:00    -1.0
1998-01-09 00:00:00+00:00    -1.0
1998-01-12 00:00:00+00:00     0.0
1998-01-13 00:00:00+00:00     1.0
1998-01-14 00:00:00+00:00     0.0
1998-01-15 00:00:00+00:00    -1.0
1998-01-16 00:00:00+00:00     0.0
1998-01-20 00:00:00+00:00     1.0
1998-01-21 00:00:00+00:00     0.0
1998-01-22 00:00:00+00:00     0.0
1998-01-23 00:00:00+00:00     0.0
1998-01-26 00:00:00+00:00     0.0
1998-01-27 00:00:00+00:00     1.0
1998-01-28 00:00:00+00:00     1.0
1998-01-29 00:00:00+00:00     0.0
1998-01-30 00:00:00+00:00     0.0
1998-02-02 00:00:00+00:00     1.0
1998-02-03 00:00:00+00:00     0.0
1998-02-04 00:00:00+00:00     0.0
1998-02-05 00:00:00+00:00     0.0
1998-02-06 00:00:00+00:00     0

INFO:root:dumping raw_pba_oc_retxeod_reteod_direod(1%)...
DEBUG:root:post_transform:                            pba_oc
id                               
1998-01-02 00:00:00+00:00     0.0
1998-01-05 00:00:00+00:00     0.0
1998-01-06 00:00:00+00:00    -1.0
1998-01-07 00:00:00+00:00    -1.0
1998-01-08 00:00:00+00:00    -1.0
1998-01-09 00:00:00+00:00    -1.0
1998-01-12 00:00:00+00:00     1.0
1998-01-13 00:00:00+00:00     1.0
1998-01-14 00:00:00+00:00     1.0
1998-01-15 00:00:00+00:00    -1.0
1998-01-16 00:00:00+00:00     1.0
1998-01-20 00:00:00+00:00     1.0
1998-01-21 00:00:00+00:00    -1.0
1998-01-22 00:00:00+00:00    -1.0
1998-01-23 00:00:00+00:00    -1.0
1998-01-26 00:00:00+00:00     0.0
1998-01-27 00:00:00+00:00     1.0
1998-01-28 00:00:00+00:00     1.0
1998-01-29 00:00:00+00:00     1.0
1998-01-30 00:00:00+00:00    -1.0
1998-02-02 00:00:00+00:00     1.0
1998-02-03 00:00:00+00:00     0.0
1998-02-04 00:00:00+00:00     0.0
1998-02-05 00:00:00+00:00     0.0
1998-02-06 00:00:00+00:00     1

INFO:root:dumping raw_pba_oc_retxeod_reteod_direod(0%)...
DEBUG:root:post_transform:                            pba_oc
id                               
1998-01-02 00:00:00+00:00     1.0
1998-01-05 00:00:00+00:00     1.0
1998-01-06 00:00:00+00:00    -1.0
1998-01-07 00:00:00+00:00    -1.0
1998-01-08 00:00:00+00:00     1.0
1998-01-09 00:00:00+00:00    -1.0
1998-01-12 00:00:00+00:00     1.0
1998-01-13 00:00:00+00:00     1.0
1998-01-14 00:00:00+00:00     1.0
1998-01-15 00:00:00+00:00    -1.0
1998-01-16 00:00:00+00:00     1.0
1998-01-20 00:00:00+00:00     1.0
1998-01-21 00:00:00+00:00    -1.0
1998-01-22 00:00:00+00:00    -1.0
1998-01-23 00:00:00+00:00     1.0
1998-01-26 00:00:00+00:00    -1.0
1998-01-27 00:00:00+00:00     1.0
1998-01-28 00:00:00+00:00     1.0
1998-01-29 00:00:00+00:00     1.0
1998-01-30 00:00:00+00:00     1.0
1998-02-02 00:00:00+00:00     1.0
1998-02-03 00:00:00+00:00     1.0
1998-02-04 00:00:00+00:00     1.0
1998-02-05 00:00:00+00:00    -1.0
1998-02-06 00:00:00+00:00     1

INFO:root:data: russell_2000_reteod_raw_pba_oc_retxeod_reteod
DEBUG:root:pre_transform:                              pba_oc
id                                 
1998-01-02 00:00:00+00:00 -0.001144
1998-01-05 00:00:00+00:00  0.001237
1998-01-06 00:00:00+00:00 -0.009061
1998-01-07 00:00:00+00:00 -0.013946
1998-01-08 00:00:00+00:00 -0.009424
1998-01-09 00:00:00+00:00 -0.029973
1998-01-12 00:00:00+00:00 -0.005013
1998-01-13 00:00:00+00:00  0.018400
1998-01-14 00:00:00+00:00  0.008341
1998-01-15 00:00:00+00:00 -0.000403
1998-01-16 00:00:00+00:00  0.010670
1998-01-20 00:00:00+00:00  0.011871
1998-01-21 00:00:00+00:00 -0.002807
1998-01-22 00:00:00+00:00 -0.008584
1998-01-23 00:00:00+00:00 -0.003261
1998-01-26 00:00:00+00:00 -0.008945
1998-01-27 00:00:00+00:00  0.003610
1998-01-28 00:00:00+00:00  0.014366
1998-01-29 00:00:00+00:00  0.007909
1998-01-30 00:00:00+00:00 -0.004491
1998-02-02 00:00:00+00:00  0.010162
1998-02-03 00:00:00+00:00  0.007780
1998-02-04 00:00:00+00:00  0.009251
1998-02-05 0

...

INFO:root:dumping raw_vol_lh_retxeod_reteod_direod(0%)...
DEBUG:root:post_transform:                            vol_lh
id                               
2005-03-18 00:00:00+00:00     1.0
2005-03-21 00:00:00+00:00     1.0
2005-03-22 00:00:00+00:00     1.0
2005-03-23 00:00:00+00:00     1.0
2005-03-24 00:00:00+00:00     1.0
2005-03-28 00:00:00+00:00    -1.0
2005-03-29 00:00:00+00:00     1.0
2005-03-30 00:00:00+00:00    -1.0
2005-03-31 00:00:00+00:00     1.0
2005-04-01 00:00:00+00:00     1.0
2005-04-04 00:00:00+00:00    -1.0
2005-04-05 00:00:00+00:00     1.0
2005-04-06 00:00:00+00:00    -1.0
2005-04-07 00:00:00+00:00    -1.0
2005-04-08 00:00:00+00:00     1.0
2005-04-11 00:00:00+00:00     1.0
2005-04-12 00:00:00+00:00    -1.0
2005-04-13 00:00:00+00:00     1.0
2005-04-14 00:00:00+00:00     1.0
2005-04-15 00:00:00+00:00     1.0
2005-04-18 00:00:00+00:00    -1.0
2005-04-19 00:00:00+00:00     1.0
2005-04-20 00:00:00+00:00     1.0
2005-04-21 00:00:00+00:00    -1.0
2005-04-22 00:00:00+00:00     1

INFO:root:data: sp_500_reteod_raw_vol_lh_retxeod_reteod
DEBUG:root:pre_transform:                              vol_lh
id                                 
1998-01-02 00:00:00+00:00  0.000000
1998-01-05 00:00:00+00:00  0.026401
1998-01-06 00:00:00+00:00  0.053457
1998-01-07 00:00:00+00:00  0.078731
1998-01-08 00:00:00+00:00  0.002243
1998-01-09 00:00:00+00:00  0.269036
1998-01-12 00:00:00+00:00 -0.084950
1998-01-13 00:00:00+00:00 -0.041682
1998-01-14 00:00:00+00:00  0.011359
1998-01-15 00:00:00+00:00  0.023558
1998-01-16 00:00:00+00:00 -0.015311
1998-01-20 00:00:00+00:00 -0.021924
1998-01-21 00:00:00+00:00  0.064589
1998-01-22 00:00:00+00:00  0.017417
1998-01-23 00:00:00+00:00  0.104531
1998-01-26 00:00:00+00:00  0.028193
1998-01-27 00:00:00+00:00  0.000894
1998-01-28 00:00:00+00:00  0.028652
1998-01-29 00:00:00+00:00  0.000903
1998-01-30 00:00:00+00:00  0.005492
1998-02-02 00:00:00+00:00  0.007435
1998-02-03 00:00:00+00:00 -0.019204
1998-02-04 00:00:00+00:00 -0.013973
1998-02-05 00:00:0

INFO:root:dumping raw_vol_lh_retxeod_reteod_direod(2%)...
DEBUG:root:post_transform:                            vol_lh
id                               
1998-01-02 00:00:00+00:00     0.0
1998-01-05 00:00:00+00:00     1.0
1998-01-06 00:00:00+00:00     1.0
1998-01-07 00:00:00+00:00     1.0
1998-01-08 00:00:00+00:00     0.0
1998-01-09 00:00:00+00:00     1.0
1998-01-12 00:00:00+00:00    -1.0
1998-01-13 00:00:00+00:00    -1.0
1998-01-14 00:00:00+00:00     1.0
1998-01-15 00:00:00+00:00     1.0
1998-01-16 00:00:00+00:00    -1.0
1998-01-20 00:00:00+00:00    -1.0
1998-01-21 00:00:00+00:00     1.0
1998-01-22 00:00:00+00:00     1.0
1998-01-23 00:00:00+00:00     1.0
1998-01-26 00:00:00+00:00     1.0
1998-01-27 00:00:00+00:00     0.0
1998-01-28 00:00:00+00:00     1.0
1998-01-29 00:00:00+00:00     0.0
1998-01-30 00:00:00+00:00     0.0
1998-02-02 00:00:00+00:00     0.0
1998-02-03 00:00:00+00:00    -1.0
1998-02-04 00:00:00+00:00    -1.0
1998-02-05 00:00:00+00:00     1.0
1998-02-06 00:00:00+00:00     0

INFO:root:dumping raw_vol_lh_retxeod_reteod_direod(1%)...
DEBUG:root:post_transform:                            vol_lh
id                               
2001-01-23 00:00:00+00:00    -1.0
2001-01-24 00:00:00+00:00     1.0
2001-01-25 00:00:00+00:00     1.0
2001-01-26 00:00:00+00:00    -1.0
2001-01-29 00:00:00+00:00    -1.0
2001-01-30 00:00:00+00:00    -1.0
2001-01-31 00:00:00+00:00    -1.0
2001-02-01 00:00:00+00:00     1.0
2001-02-02 00:00:00+00:00     1.0
2001-02-05 00:00:00+00:00    -1.0
2001-02-06 00:00:00+00:00    -1.0
2001-02-07 00:00:00+00:00    -1.0
2001-02-08 00:00:00+00:00     0.0
2001-02-09 00:00:00+00:00     1.0
2001-02-12 00:00:00+00:00    -1.0
2001-02-13 00:00:00+00:00     1.0
2001-02-14 00:00:00+00:00     0.0
2001-02-15 00:00:00+00:00    -1.0
2001-02-16 00:00:00+00:00    -1.0
2001-02-20 00:00:00+00:00     1.0
2001-02-21 00:00:00+00:00     1.0
2001-02-22 00:00:00+00:00     1.0
2001-02-23 00:00:00+00:00     1.0
2001-02-26 00:00:00+00:00    -1.0
2001-02-27 00:00:00+00:00     1

INFO:root:dumping raw_vol_lh_retxeod_reteod_direod(0%)...
DEBUG:root:post_transform:                            vol_lh
id                               
2006-05-05 00:00:00+00:00     1.0
2006-05-08 00:00:00+00:00    -1.0
2006-05-09 00:00:00+00:00     1.0
2006-05-10 00:00:00+00:00     1.0
2006-05-11 00:00:00+00:00     1.0
2006-05-12 00:00:00+00:00     1.0
2006-05-15 00:00:00+00:00     1.0
2006-05-16 00:00:00+00:00     1.0
2006-05-17 00:00:00+00:00     1.0
2006-05-18 00:00:00+00:00     1.0
2006-05-19 00:00:00+00:00     1.0
2006-05-22 00:00:00+00:00     1.0
2006-05-23 00:00:00+00:00     1.0
2006-05-24 00:00:00+00:00     1.0
2006-05-25 00:00:00+00:00    -1.0
2006-05-26 00:00:00+00:00    -1.0
2006-05-30 00:00:00+00:00     1.0
2006-05-31 00:00:00+00:00    -1.0
2006-06-01 00:00:00+00:00    -1.0
2006-06-02 00:00:00+00:00     1.0
2006-06-05 00:00:00+00:00     1.0
2006-06-06 00:00:00+00:00     1.0
2006-06-07 00:00:00+00:00     1.0
2006-06-08 00:00:00+00:00     1.0
2006-06-09 00:00:00+00:00     1

## Test Disk Version

In [9]:
au = ['dir', 'direod']
actual_dg, actual_cs = list_get_dict(dg, au), list_get_dict(cs2, au)
actual_paths, actual_recs, actual_dfs = DataAPI.load_from_dg(actual_dg, actual_cs)
logging.debug('actual_paths[0] ' +str(actual_paths[0]))
logging.debug('actual_paths[-1] ' +str(actual_paths[-1]))

DEBUG:root:actual_paths[0] ['dow_jones', 'direod', 'raw_pba_oc_retxeod_reteod_direod', 'raw_pba_oc_retxeod_reteod_direod(0%)']
DEBUG:root:actual_paths[-1] ['russell_2000', 'direod', 'raw_vol_lh_retxeod_reteod_direod', 'raw_vol_lh_retxeod_reteod_direod(2%)']


In [10]:
for key_chain in actual_paths:
    actual_rec, actual_df = list_get_dict(actual_recs, key_chain), list_get_dict(actual_dfs, key_chain)
    logging.debug(str(key_chain))
    logging.debug(actual_df)

DEBUG:root:['dow_jones', 'direod', 'raw_pba_oc_retxeod_reteod_direod', 'raw_pba_oc_retxeod_reteod_direod(0%)']
DEBUG:root:                           pba_oc
id                               
1998-01-02 00:00:00+00:00     1.0
1998-01-03 00:00:00+00:00     NaN
1998-01-04 00:00:00+00:00     NaN
1998-01-05 00:00:00+00:00     1.0
1998-01-06 00:00:00+00:00    -1.0
1998-01-07 00:00:00+00:00    -1.0
1998-01-08 00:00:00+00:00    -1.0
1998-01-09 00:00:00+00:00    -1.0
1998-01-10 00:00:00+00:00     NaN
1998-01-11 00:00:00+00:00     NaN
1998-01-12 00:00:00+00:00     1.0
1998-01-13 00:00:00+00:00     1.0
1998-01-14 00:00:00+00:00     1.0
1998-01-15 00:00:00+00:00    -1.0
1998-01-16 00:00:00+00:00     1.0
1998-01-17 00:00:00+00:00     NaN
1998-01-18 00:00:00+00:00     NaN
1998-01-19 00:00:00+00:00     NaN
1998-01-20 00:00:00+00:00     1.0
1998-01-21 00:00:00+00:00    -1.0
1998-01-22 00:00:00+00:00    -1.0
1998-01-23 00:00:00+00:00    -1.0
1998-01-24 00:00:00+00:00     NaN
1998-01-25 00:00:00+00:00   

DEBUG:root:['sp_500', 'direod', 'raw_pba_oc_retxeod_reteod_direod', 'raw_pba_oc_retxeod_reteod_direod(0%)']
DEBUG:root:                           pba_oc
id                               
1998-01-02 00:00:00+00:00     1.0
1998-01-03 00:00:00+00:00     NaN
1998-01-04 00:00:00+00:00     NaN
1998-01-05 00:00:00+00:00     1.0
1998-01-06 00:00:00+00:00    -1.0
1998-01-07 00:00:00+00:00    -1.0
1998-01-08 00:00:00+00:00    -1.0
1998-01-09 00:00:00+00:00    -1.0
1998-01-10 00:00:00+00:00     NaN
1998-01-11 00:00:00+00:00     NaN
1998-01-12 00:00:00+00:00     1.0
1998-01-13 00:00:00+00:00     1.0
1998-01-14 00:00:00+00:00     1.0
1998-01-15 00:00:00+00:00    -1.0
1998-01-16 00:00:00+00:00     1.0
1998-01-17 00:00:00+00:00     NaN
1998-01-18 00:00:00+00:00     NaN
1998-01-19 00:00:00+00:00     NaN
1998-01-20 00:00:00+00:00     1.0
1998-01-21 00:00:00+00:00    -1.0
1998-01-22 00:00:00+00:00    -1.0
1998-01-23 00:00:00+00:00    -1.0
1998-01-24 00:00:00+00:00     NaN
1998-01-25 00:00:00+00:00     N

DEBUG:root:['nasdaq_100', 'direod', 'raw_pba_oc_retxeod_reteod_direod', 'raw_pba_oc_retxeod_reteod_direod(0%)']
DEBUG:root:                           pba_oc
id                               
1998-01-02 00:00:00+00:00     1.0
1998-01-03 00:00:00+00:00     NaN
1998-01-04 00:00:00+00:00     NaN
1998-01-05 00:00:00+00:00     1.0
1998-01-06 00:00:00+00:00    -1.0
1998-01-07 00:00:00+00:00    -1.0
1998-01-08 00:00:00+00:00     1.0
1998-01-09 00:00:00+00:00    -1.0
1998-01-10 00:00:00+00:00     NaN
1998-01-11 00:00:00+00:00     NaN
1998-01-12 00:00:00+00:00     1.0
1998-01-13 00:00:00+00:00     1.0
1998-01-14 00:00:00+00:00     1.0
1998-01-15 00:00:00+00:00    -1.0
1998-01-16 00:00:00+00:00     1.0
1998-01-17 00:00:00+00:00     NaN
1998-01-18 00:00:00+00:00     NaN
1998-01-19 00:00:00+00:00     NaN
1998-01-20 00:00:00+00:00     1.0
1998-01-21 00:00:00+00:00    -1.0
1998-01-22 00:00:00+00:00    -1.0
1998-01-23 00:00:00+00:00     1.0
1998-01-24 00:00:00+00:00     NaN
1998-01-25 00:00:00+00:00  

DEBUG:root:['russell_2000', 'direod', 'raw_pba_oc_retxeod_reteod_direod', 'raw_pba_oc_retxeod_reteod_direod(0%)']
DEBUG:root:                           pba_oc
id                               
1998-01-02 00:00:00+00:00    -1.0
1998-01-03 00:00:00+00:00     NaN
1998-01-04 00:00:00+00:00     NaN
1998-01-05 00:00:00+00:00     1.0
1998-01-06 00:00:00+00:00    -1.0
1998-01-07 00:00:00+00:00    -1.0
1998-01-08 00:00:00+00:00    -1.0
1998-01-09 00:00:00+00:00    -1.0
1998-01-10 00:00:00+00:00     NaN
1998-01-11 00:00:00+00:00     NaN
1998-01-12 00:00:00+00:00    -1.0
1998-01-13 00:00:00+00:00     1.0
1998-01-14 00:00:00+00:00     1.0
1998-01-15 00:00:00+00:00    -1.0
1998-01-16 00:00:00+00:00     1.0
1998-01-17 00:00:00+00:00     NaN
1998-01-18 00:00:00+00:00     NaN
1998-01-19 00:00:00+00:00     NaN
1998-01-20 00:00:00+00:00     1.0
1998-01-21 00:00:00+00:00    -1.0
1998-01-22 00:00:00+00:00    -1.0
1998-01-23 00:00:00+00:00    -1.0
1998-01-24 00:00:00+00:00     NaN
1998-01-25 00:00:00+00:00

DEBUG:root:['dow_jones', 'direod', 'raw_pba_oa_retxeod_reteod_direod', 'raw_pba_oa_retxeod_reteod_direod(0%)']
DEBUG:root:                           pba_oa
id                               
1998-01-02 00:00:00+00:00     1.0
1998-01-03 00:00:00+00:00     NaN
1998-01-04 00:00:00+00:00     NaN
1998-01-05 00:00:00+00:00     1.0
1998-01-06 00:00:00+00:00    -1.0
1998-01-07 00:00:00+00:00    -1.0
1998-01-08 00:00:00+00:00    -1.0
1998-01-09 00:00:00+00:00    -1.0
1998-01-10 00:00:00+00:00     NaN
1998-01-11 00:00:00+00:00     NaN
1998-01-12 00:00:00+00:00     1.0
1998-01-13 00:00:00+00:00     1.0
1998-01-14 00:00:00+00:00     1.0
1998-01-15 00:00:00+00:00    -1.0
1998-01-16 00:00:00+00:00     1.0
1998-01-17 00:00:00+00:00     NaN
1998-01-18 00:00:00+00:00     NaN
1998-01-19 00:00:00+00:00     NaN
1998-01-20 00:00:00+00:00     1.0
1998-01-21 00:00:00+00:00    -1.0
1998-01-22 00:00:00+00:00    -1.0
1998-01-23 00:00:00+00:00    -1.0
1998-01-24 00:00:00+00:00     NaN
1998-01-25 00:00:00+00:00   

...

DEBUG:root:['dow_jones', 'direod', 'raw_vol_lh_retxeod_reteod_direod', 'raw_vol_lh_retxeod_reteod_direod(0%)']
DEBUG:root:                           vol_lh
id                               
2005-03-18 00:00:00+00:00     1.0
2005-03-19 00:00:00+00:00     NaN
2005-03-20 00:00:00+00:00     NaN
2005-03-21 00:00:00+00:00     1.0
2005-03-22 00:00:00+00:00     1.0
2005-03-23 00:00:00+00:00     1.0
2005-03-24 00:00:00+00:00     1.0
2005-03-25 00:00:00+00:00     NaN
2005-03-26 00:00:00+00:00     NaN
2005-03-27 00:00:00+00:00     NaN
2005-03-28 00:00:00+00:00    -1.0
2005-03-29 00:00:00+00:00     1.0
2005-03-30 00:00:00+00:00    -1.0
2005-03-31 00:00:00+00:00     1.0
2005-04-01 00:00:00+00:00     1.0
2005-04-02 00:00:00+00:00     NaN
2005-04-03 00:00:00+00:00     NaN
2005-04-04 00:00:00+00:00    -1.0
2005-04-05 00:00:00+00:00     1.0
2005-04-06 00:00:00+00:00    -1.0
2005-04-07 00:00:00+00:00    -1.0
2005-04-08 00:00:00+00:00     1.0
2005-04-09 00:00:00+00:00     NaN
2005-04-10 00:00:00+00:00   

DEBUG:root:['sp_500', 'direod', 'raw_vol_lh_retxeod_reteod_direod', 'raw_vol_lh_retxeod_reteod_direod(0%)']
DEBUG:root:                           vol_lh
id                               
1998-01-02 00:00:00+00:00    -1.0
1998-01-03 00:00:00+00:00     NaN
1998-01-04 00:00:00+00:00     NaN
1998-01-05 00:00:00+00:00     1.0
1998-01-06 00:00:00+00:00     1.0
1998-01-07 00:00:00+00:00     1.0
1998-01-08 00:00:00+00:00     1.0
1998-01-09 00:00:00+00:00     1.0
1998-01-10 00:00:00+00:00     NaN
1998-01-11 00:00:00+00:00     NaN
1998-01-12 00:00:00+00:00    -1.0
1998-01-13 00:00:00+00:00    -1.0
1998-01-14 00:00:00+00:00     1.0
1998-01-15 00:00:00+00:00     1.0
1998-01-16 00:00:00+00:00    -1.0
1998-01-17 00:00:00+00:00     NaN
1998-01-18 00:00:00+00:00     NaN
1998-01-19 00:00:00+00:00     NaN
1998-01-20 00:00:00+00:00    -1.0
1998-01-21 00:00:00+00:00     1.0
1998-01-22 00:00:00+00:00     1.0
1998-01-23 00:00:00+00:00     1.0
1998-01-24 00:00:00+00:00     NaN
1998-01-25 00:00:00+00:00     N

DEBUG:root:['nasdaq_100', 'direod', 'raw_vol_lh_retxeod_reteod_direod', 'raw_vol_lh_retxeod_reteod_direod(0%)']
DEBUG:root:                           vol_lh
id                               
2001-01-23 00:00:00+00:00    -1.0
2001-01-24 00:00:00+00:00     1.0
2001-01-25 00:00:00+00:00     1.0
2001-01-26 00:00:00+00:00    -1.0
2001-01-27 00:00:00+00:00     NaN
2001-01-28 00:00:00+00:00     NaN
2001-01-29 00:00:00+00:00    -1.0
2001-01-30 00:00:00+00:00    -1.0
2001-01-31 00:00:00+00:00    -1.0
2001-02-01 00:00:00+00:00     1.0
2001-02-02 00:00:00+00:00     1.0
2001-02-03 00:00:00+00:00     NaN
2001-02-04 00:00:00+00:00     NaN
2001-02-05 00:00:00+00:00    -1.0
2001-02-06 00:00:00+00:00    -1.0
2001-02-07 00:00:00+00:00    -1.0
2001-02-08 00:00:00+00:00     1.0
2001-02-09 00:00:00+00:00     1.0
2001-02-10 00:00:00+00:00     NaN
2001-02-11 00:00:00+00:00     NaN
2001-02-12 00:00:00+00:00    -1.0
2001-02-13 00:00:00+00:00     1.0
2001-02-14 00:00:00+00:00    -1.0
2001-02-15 00:00:00+00:00  

DEBUG:root:['russell_2000', 'direod', 'raw_vol_lh_retxeod_reteod_direod', 'raw_vol_lh_retxeod_reteod_direod(0%)']
DEBUG:root:                           vol_lh
id                               
2006-05-05 00:00:00+00:00     1.0
2006-05-06 00:00:00+00:00     NaN
2006-05-07 00:00:00+00:00     NaN
2006-05-08 00:00:00+00:00    -1.0
2006-05-09 00:00:00+00:00     1.0
2006-05-10 00:00:00+00:00     1.0
2006-05-11 00:00:00+00:00     1.0
2006-05-12 00:00:00+00:00     1.0
2006-05-13 00:00:00+00:00     NaN
2006-05-14 00:00:00+00:00     NaN
2006-05-15 00:00:00+00:00     1.0
2006-05-16 00:00:00+00:00     1.0
2006-05-17 00:00:00+00:00     1.0
2006-05-18 00:00:00+00:00     1.0
2006-05-19 00:00:00+00:00     1.0
2006-05-20 00:00:00+00:00     NaN
2006-05-21 00:00:00+00:00     NaN
2006-05-22 00:00:00+00:00     1.0
2006-05-23 00:00:00+00:00     1.0
2006-05-24 00:00:00+00:00     1.0
2006-05-25 00:00:00+00:00    -1.0
2006-05-26 00:00:00+00:00    -1.0
2006-05-27 00:00:00+00:00     NaN
2006-05-28 00:00:00+00:00

[4257 rows x 1 columns]
